In [41]:
import pandas as pd
import numpy as np
import os
import itertools
from openpyxl import load_workbook
from IPython.display import display

In [42]:
# File to be reading
file = 'NF 80.xlsx'
file_path = f"podklady/{file}"

# Sheet Name
sheet_name = '4 KŘ PSK'

# Which row should be skipped
skiprows = 1

# Till wich row shold program read rows
nrows = 10

# Which columns shoud program read
usecols = 'A:AH'
#usecols = 'J:P'

# barva
#barva = "weiss"
#barva = "dekor"

# nová tabulka - jednotlivé položky
material = ["?"]
breite = []
hohe = []
preis = []
preismatrix = [f"NATURFENSTER 80{barva}"]
product_key = ["PSK 4-Tlg"]

# Cesta k novému souboru
path = f"prevedene_tabulky/upraveno_{file}"
sheet = f"{product_key[0]} {preismatrix[0][-5:]}"

# # Finální tabulka
new_data = []

In [43]:
kec = "blah"
data = [f"some text {kec}"]
#print(data[0])

In [44]:
# Read data from file and specific sheet. Read data from a specific range
pd.set_option("display.max_row", None)
df = pd.read_excel(file_path,sheet_name=sheet_name, header = None, skiprows = skiprows, nrows=nrows,  usecols= usecols)
df.columns = range(df.columns.size)
#display(df)

In [45]:
df = df.replace([np.nan, 'V/Š'], [0, 0], regex=True)
df = df.astype('int')
#display(df)

In [46]:
# Počet řádku v tabulce
table_row = len(df)-1
#print(table_row)

In [47]:
# Počet sloupců v tabulce
table_column = len(df.columns)-1
#print(table_column)

In [48]:
# Hohe - Výška
for index, row in df.iterrows():
    hohe.append(row[0])
hohe.pop(0)
hohe = [el for el in hohe for i in range(table_column)]
#print(hohe)

In [49]:
# Breite - Šířka
for column in df:
    breite.append(df[column][0])
breite.pop(0)

breite = breite*table_row
#print(breite)

In [50]:
# Ceny v listu
df_list = df.values.tolist()
del df_list[0]

for i in df_list:
    del i[0]

preis = list(itertools.chain.from_iterable(df_list))

In [51]:
lst = [material *len(preis),breite, hohe, preis, preismatrix*len(preis), product_key*len(preis)]

# Vytváření tabulky z listů uvedených v lst
table = pd.DataFrame(lst)

# Otočení tabulky
final_table = table.transpose()

# Pojmenování jednotlivých sloupců
final_table.columns = ["Material", "Breite", "Hohe", "Preis", "Preismatrix", "Product Key"]

# Odstranění nulových hodnot podle sloupce "Preis" 
final_table = final_table[final_table.Preis != 0]
#final_table

In [52]:
if os.path.exists(path):
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book
    final_table.to_excel(writer, sheet_name = sheet, index=False)
    writer.close()
else:
    final_table.to_excel(path, sheet_name = sheet, index=False)